In [ ]:
from polymarket_analysis.data.binance import Binance

from datetime import datetime

from_date = datetime.fromisoformat("2025-06-01")
to_date = datetime.fromisoformat("2025-06-05")



In [ ]:
# btc_df = Binance.load_bitcon_5min(
#     from_date=from_date, 
#     to_date=to_date
# )

In [ ]:
from polymarket_analysis.analytics.gbm_param_estimator import GBMParameterEstimator

# Initialize estimator
gbm = GBMParameterEstimator(symbol="BTC-USD", period="1y")

gbm.fetch_data()
# Fetch data and estimate parameters
gbm.calculate_returns()
params = gbm.estimate_parameters()

# Print parameter summary
summary = gbm.get_parameters_summary()
print("\n=== GBM Parameter Estimation for Bitcoin ===")
print(f"Symbol: {summary['symbol']}")
print(f"Period: {summary['period']}")
print(f"Observations: {summary['n_observations']}")
print(f"Current Price: ${summary['current_price']:.2f}")
print(f"\nEstimated Parameters:")
print(f"Annualized Drift (μ): {summary['annualized_drift']:.4f} ({summary['annualized_drift']:.2%})")
print(f"Annualized Volatility (σ): {summary['annualized_volatility']:.4f} ({summary['annualized_volatility']:.2%})")
print(f"Sharpe Ratio: {summary['sharpe_ratio']:.4f}")

# Example probability analysis
current_price = summary['current_price']
target_price = current_price * 1.1  # 10% increase
days_ahead = 30

prob_analysis = gbm.probability_analysis(target_price, current_price, days_ahead)

print(f"\n=== Probability Analysis ===")
print(f"Current Price: ${prob_analysis['current_price']:.2f}")
print(f"Target Price: ${prob_analysis['target_price']:.2f} ({((target_price/current_price-1)*100):+.1f}%)")
print(f"Time Horizon: {days_ahead} days")
print(f"Probability above target: {prob_analysis['probability_above_target']:.2%}")
print(f"Expected price: ${prob_analysis['expected_price']:.2f}")

print(f"\nPrice Percentiles in {days_ahead} days:")
for pct, price in prob_analysis['price_percentiles'].items():
    print(f"{pct}: ${price:.2f}")
    

In [ ]:
from polymarket_analysis.analytics.gbm_backtester import GBMBacktester

# Create and run backtest
backtester = GBMBacktester("BTC-USD", lookback_days=365, test_days=180)
backtester.fetch_extended_data()


In [ ]:
results = backtester.run_backtest(forecast_horizons=[1, 2, 7, 14, 30], 
                                  target_changes=[0.01, 0.02, 0.05, 0.1, 0.1])
backtester.generate_simple_report()


In [ ]:
# Reload the fixed backtester module
import importlib
from polymarket_analysis.analytics import gbm_backtester
importlib.reload(gbm_backtester)

# Create a fresh backtester instance with the fixes
backtester_fixed = gbm_backtester.GBMBacktester("BTC-USD", lookback_days=400, test_days=400)
backtester_fixed.fetch_extended_data()

In [ ]:
results_fixed = backtester_fixed.run_backtest(
    forecast_horizons=[1, 2, 7, 14, 30],
    target_changes=[0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0, -0.02, -0.05, -0.1]
)

In [ ]:
# Now try the plotting (should work without KeyError)
calibration_fig = backtester_fixed.plot_relative_errors()
calibration_fig.show()
print("✓ Plotting successful!")

# Show calibration data
calibration_data = backtester_fixed.analyze_calibration()
print("\nCalibration Data:")
print(calibration_data.round(3))